Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from yeast8model import Yeast8Model, compare_fluxes, compare_ablation_times, heatmap_ablation_grid

# Construct models of cells of interest, optimise

In [ ]:
glc_exch_rate = 16.89

wt = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.solution = wt.optimize()

BY4741 = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
BY4741.make_auxotroph("BY4741")
BY4741.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
BY4741.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
BY4741.solution = BY4741.optimize()

BY4742 = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
BY4742.make_auxotroph("BY4742")
BY4742.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
BY4742.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
BY4742.solution = BY4742.optimize()

zwf1 = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
zwf1.make_auxotroph("BY4741")
zwf1.knock_out_list(["YNL241C"])
zwf1.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
zwf1.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
zwf1.solution = zwf1.optimize()

tsa2 = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
tsa2.make_auxotroph("BY4742")
tsa2.knock_out_list(["YDR453C"])
tsa2.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
tsa2.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
tsa2.solution = tsa2.optimize()

# Ablate

TODO: For loop all this?  It's repetitive.

## Wild type

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type')
plt.show()

## BY4741

In [ ]:
BY4741.ablation_result = BY4741.ablate()

fig, ax = plt.subplots()
BY4741.ablation_barplot(ax)
ax.set_title('BY4741 (in supplemented media)')
plt.show()

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt.ablation_result, BY4741.ablation_result, ax_fc)

## BY4742

In [ ]:
BY4742.ablation_result = BY4742.ablate()

fig, ax = plt.subplots()
BY4742.ablation_barplot(ax)
ax.set_title('BY4742 (in supplemented media)')
plt.show()

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt.ablation_result, BY4742.ablation_result, ax_fc)

In [ ]:
zwf1.ablation_result = zwf1.ablate()
fig, ax = plt.subplots()
zwf1.ablation_barplot(ax)
ax.set_title('zwf1$\Delta$ BY4741 (in supplemented media)')
plt.show()

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(BY4741.ablation_result, zwf1.ablation_result, ax_fc)

In [ ]:
tsa2.ablation_result = tsa2.ablate()
fig, ax = plt.subplots()
tsa2.ablation_barplot(ax)
ax.set_title('tsa2$\Delta$ BY4742 (in supplemented media)')
plt.show()

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(BY4742.ablation_result, tsa2.ablation_result, ax_fc)

In [ ]:
wt.ablation_result

In [ ]:
BY4741.ablation_result

In [ ]:
BY4742.ablation_result

In [ ]:
zwf1.ablation_result

In [ ]:
tsa2.ablation_result

In [ ]:
wt_ablation_result_orig = wt.ablation_result.copy()

## Add amino acids to minimal media

In [ ]:
amino_exch_list = [
    'r_1873', 'r_1879', 'r_1880', 'r_1881', 'r_1883',
    'r_1889', 'r_1891', 'r_1810', 'r_1893', 'r_1897',
    'r_1899', 'r_1900', 'r_1902', 'r_1903', 'r_1904',
    'r_1906', 'r_1911', 'r_1912', 'r_1913', 'r_1914',
]

wt.reset_to_file()
wt.unrestrict_growth()
wt.add_media_components(amino_exch_list)
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

In [ ]:
for reaction_id in wt.model.medium.keys():
    print(wt.model.reactions.get_by_id(reaction_id).name)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type, with all amino acids supplemented')
plt.show()

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

> You can see that it takes less time to make proteins.

## Add nucleotides to minimal media

In [ ]:
nucl_exch_list = [
    'r_1639', 'r_1705', 'r_1818', 'r_2090'
]

wt.reset_to_file()
wt.unrestrict_growth()
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.add_media_components(nucl_exch_list)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type, with all NTPs supplemented')
plt.show()

In [ ]:
wt.ablation_result

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

## Add deoxyribonucleotides to media

In [ ]:
deoxnucl_exch_list = [
    'r_1643', 'r_1702', 'r_1820', 'r_2073'
]

wt.reset_to_file()
wt.unrestrict_growth()
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.add_media_components(deoxnucl_exch_list)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type, with all dNTPs supplemented')
plt.show()

In [ ]:
wt.ablation_result

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

## Carbon sources

Pyruvate

In [ ]:
wt.reset_to_file()
wt.unrestrict_growth()
wt.add_media_components(['r_2033'])
# Set uptake rate (under saturation) to get a growth rate consistent with experiments.
wt.model.reactions.get_by_id('r_2033').bounds = (-5.476, 0)
wt.remove_media_components(['r_1714', 'r_1714_REV'])
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type, with pyruvate as carbon source')
plt.show()

In [ ]:
wt.ablation_result

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

Glucose limitation

In [ ]:
glc_exch_rate = 1.329
# Simulates 10 mg/L by assuming a 3-hour doubling time
# and using the saturation curve at the end of this notebook

wt.reset_to_file()
wt.unrestrict_growth()
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
ax.set_title('Wild type, with glucose limitation')
plt.show()

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

# Compare fluxes

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, zwf1)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, tsa2)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
for reaction_id in BY4741.model.medium.keys():
    print(BY4741.model.reactions.get_by_id(reaction_id).name)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, wt)

In [ ]:
diff_fluxes_sorted

# Effect of carbon source uptake on growth rate

## Glucose

Optimise wt with glucose unconstrained.

In [ ]:
wt.reset_to_file()
wt.unrestrict_growth()
wt.solution = wt.optimize()

Get saturated glucose uptake:

In [ ]:
wt.solution['r_1714_REV']

> You should get 16.9 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup>.  This agrees with Elsemman et al. (2022): they predict saturation at 18.6 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup> and report a range of 16 ~ 19 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup> from the literature (Blank et al., 2004).

Sweep, across all strains.

In [ ]:
wt_suppl = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
wt_suppl.add_media_components([
    "r_1893",
    "r_1899",
    "r_1902",
    "r_2090",
    "r_1893_REV",
    "r_1899_REV",
    "r_1902_REV",
    "r_2090_REV",
])

In [ ]:
# positive values because i want it increasing in a plot
glc_exch_rates = np.linspace(0, 18.6, 100)

wt_growthrate = []
wt_suppl_growthrate = []
BY4741_growthrate = []
zwf1_growthrate = []
tsa2_growthrate = []

def get_gr_from_glc_rate(ymodel, glc_exch_rate, growthratelist):
    # negative due to FBA conventions re exchange reactions
    ymodel.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
    # positive due to FBA conventions re reversible reaction
    ymodel.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
    ymodel.solution = ymodel.optimize()
    growthratelist.append(ymodel.solution.fluxes["r_2111"])

for glc_exch_rate in glc_exch_rates:
    get_gr_from_glc_rate(wt, glc_exch_rate, wt_growthrate)
    get_gr_from_glc_rate(wt_suppl, glc_exch_rate, wt_suppl_growthrate)
    get_gr_from_glc_rate(BY4741, glc_exch_rate, BY4741_growthrate)
    get_gr_from_glc_rate(zwf1, glc_exch_rate, zwf1_growthrate)
    get_gr_from_glc_rate(tsa2, glc_exch_rate, tsa2_growthrate)

print('optimisations done')

In [ ]:
fig, ax = plt.subplots()
ax.plot(glc_exch_rates, wt_growthrate, label='prototroph (wild type)')
#ax.plot(glc_exch_rates, wt_suppl_growthrate, label='prototroph, in BY4741 supplemented media')
ax.plot(glc_exch_rates, BY4741_growthrate, label='BY4741, in supplemented media')
ax.plot(glc_exch_rates, zwf1_growthrate, label='BY4741 zwf1, in supplemented media')
ax.plot(glc_exch_rates, tsa2_growthrate, label='BY4742 tsa2, in supplemented media')
ax.set_xlim((0,20))
ax.set_xlabel('Glucose exchange rate (mmol/gDW/h)')
ax.set_ylabel('Growth rate (/h)')
ax.set_title('Effect of glucose exchange rate on growth rate')
ax.legend()

> Results from wild-type are similar to with Elsemman et al. (2022), figure 2B. Saturation (or near it) seems to be reached at about 8 mmol/gDW/h.

Alternatively, fix growth rate and optimise to get glucose uptake rate. (doesn't work)

In [ ]:
#wt.model.objective = {wt.model.reactions.get_by_id("r_1714"): +1}

growthrates = np.linspace(0, 2, 500)

wt_glc_rates = []

for growthrate in growthrates:
    wt.model.reactions.get_by_id(wt.growth_id).bounds = (growthrate, growthrate)
    wt.model.reactions.get_by_id("r_1714").bounds = (-1000, 0)
    wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, 1000)
    wt.solution = wt.model.optimize()
    wt_glc_rates.append(wt.solution.fluxes["r_1714_REV"])
    
print('optimisations done')

In [ ]:
plt.plot(growthrates, wt_glc_rates)

In [ ]:
doubling_times = np.log(2) / np.array(wt_growthrate)

fig, ax = plt.subplots()
ax.plot(glc_exch_rates, doubling_times)
ax.set_xlim((1,8))
ax.set_ylim((0,5))
ax.set_xlabel('Glucose exchange rate (mmol/gDW/h)')
ax.set_ylabel('Doubling time (h)')
ax.set_title('Effect of glucose exchange rate on doubling time')

Get glucose exchange rate that results in a certain growth rate...

This is based on linear interpolation.  The implied function takes growth rate as the independent value and glucose exchange rate as the dependent value (i.e. the inverse function of the saturation curve).

If necessary, get growth rate from doubling time.  Based on: $T = \frac{\log 2}{r}$, where $T$ is the doubling time and $r$ is the growth rate.

In [ ]:
# Get growth rate from doubling time
dt = 3
np.log(2)/dt

In [ ]:
# Get exchange rate
np.interp(0.231, wt_growthrate, glc_exch_rates)

## Pyruvate

In [ ]:
wt.reset_to_file()
wt.unrestrict_growth()
wt.add_media_components(['r_2033'])
wt.remove_media_components(['r_1714', 'r_1714_REV'])
wt.solution = wt.optimize()

Saturated uptake

In [ ]:
wt.solution['r_2033']

Sweep

In [ ]:
# positive values because i want it increasing in a plot
pyr_exch_rates = np.linspace(0, 19.2, 100)

wt_growthrate = []
BY4741_growthrate = []
zwf1_growthrate = []
tsa2_growthrate = []

def get_gr_from_pyr_rate(ymodel, growthratelist):
    # negative due to FBA conventions re exchange reactions
    ymodel.model.reactions.get_by_id("r_2033").bounds = (-pyr_exch_rate, 0)
    # no glucose
    ymodel.remove_media_components(['r_1714', 'r_1714_REV'])
    ymodel.solution = ymodel.optimize()
    growthratelist.append(ymodel.solution.fluxes["r_2111"])

for pyr_exch_rate in pyr_exch_rates:
    get_gr_from_pyr_rate(wt, wt_growthrate)
    get_gr_from_pyr_rate(BY4741, BY4741_growthrate)
    get_gr_from_pyr_rate(zwf1, zwf1_growthrate)
    get_gr_from_pyr_rate(tsa2, tsa2_growthrate)

print('optimisations done')

In [ ]:
fig, ax = plt.subplots()
ax.plot(pyr_exch_rates, wt_growthrate, label='prototroph (wild type)')
ax.plot(pyr_exch_rates, BY4741_growthrate, label='BY4741, in supplemented media')
ax.plot(pyr_exch_rates, zwf1_growthrate, label='BY4741 zwf1, in supplemented media')
ax.plot(pyr_exch_rates, tsa2_growthrate, label='BY4742 tsa2, in supplemented media')
ax.set_xlim((0,20))
ax.set_xlabel('Pyruvate exchange rate (mmol/gDW/h)')
ax.set_ylabel('Growth rate (/h)')
ax.set_title('Effect of pyruvate exchange rate on growth rate')
ax.legend()

In [ ]:
# Get exchange rate
wt_growthrate = ymodel_growthrates[0]
np.interp(0.173, wt_growthrate, pyr_exch_rates)

# Grid

TODO: Move this to another notebook.

TODO: Make sure that it isn't actually repeating the same thing 25 times, as the ratio is the same everywhere.

In [ ]:
exch_rate_dict = {
    "r_1714": np.linspace(0, 18, 5), # glucose
    "r_1654": np.linspace(0, 18, 5), # ammonium
}
ra, la = wt.ablation_grid(exch_rate_dict)

In [ ]:
fig, ax = plt.subplots()
heatmap_ablation_grid(ra, exch_rate_dict, ax)
plt.show()

In [ ]:
ra